In [ ]:
!conda install -y -c conda-forge dlib

In [ ]:
!pip install sk-video

In [ ]:
!conda install ffmpeg -y

In [1]:
import numpy as np
import dlib
import cv2
from skvideo.io import vread
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import os
import pandas as pd
import concurrent.futures
import time

In [2]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
def get_rect(shape):
    rw = 0
    rh = 0
    rx = 65535
    ry = 65535
    for (x,y) in shape:
        rw = max(rw,x)
        rh = max(rh,y)
        rx = min(rx,x)
        ry = min(ry,y)
    return (rx,ry,rw-rx,rh-ry)

def shape_to_np(shape, dtype="int"):
    
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)
        
    return coords

In [4]:
def read_video(path):
    
    #print("loading: " + path)
    try:
        video = vread(path)
        frames = np.array([cv2.cvtColor(frame,cv2.COLOR_RGB2BGR) for frame in video])
        mouth_video = get_frames_mouth(frames)
        mouth_videoo = []
        for frame in mouth_video:
            mouth_videoo.append(frame)
            mouth_videoo.append(frame)
        mouth_video = mouth_videoo

        return np.array(mouth_video)/255.0
    except:
        return None

In [5]:
def get_frames_mouth(frames):
    
    global detector,predictor
    mouth_frames = []
    for frame_img in frames:
        try:
            rects = detector(cv2.resize(frame_img, None, fx=0.5, fy=0.5) , 1)
            rects[0] = dlib.scale_rect(rects[0],1/0.5)
            shape = predictor(image=frame_img,box=rects[0])
            shape = shape_to_np(shape)
            shape = [shape[x] for x in range(48,68)]
            (x, y, w, h) = get_rect(shape)
            for s in shape:
                s[0] -= x
                s[1] -= y
            mouth = frame_img[y:y+h, x:x+w]
            mouth = cv2.resize(mouth ,(100,50))
            mouth_frames.append(mouth)
        except:
            return None
    return mouth_frames

In [ ]:
# JUST to CHECK
check = read_video('grid_corpus/video/s1/prwq3s.mpg')
print(check.shape)

In [10]:
ids = []
paths = []
output_paths = []

for speaker in os.listdir('grid_corpus/video/'):
    if speaker in ['s21']:
        continue
    for file in os.listdir('grid_corpus/video/'+speaker+'/'):
        file_path = 'grid_corpus/video/'+speaker+'/'+file
        ids.append(speaker+'_'+file[:-4])
        paths.append(file_path)
        output_paths.append('x_data/'+speaker+'_'+file[:-4]+'.npy')

#print(len(ids)) #  JUST to CHECK
#st = time.time()

def multiprocessing_func(file_path,output_path):
    if output_path not in os.listdir('x_data'):
        try:
            if file_path[-4:] != '.mpg':
                return 
            video = read_video(file_path)
            if video is None:
                print('error in ',file_path)
            np.save(output_path, video)
        except:
            print('Error in ',file_path)

with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(multiprocessing_func, paths, output_paths)
#print('Time for '+speaker , time.time() - st)
print('Done')

33033
Done
